<a href="https://colab.research.google.com/github/Jaewon45/ESAA_2022/blob/main/1121_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 08. 파이썬 추천 시스템 패키지 - Surprise
### Surprise 패키지 소개
- 파이썬 기반의 추천 시스템 구축을 위한 전용 패키지 중 하나로 파이썬 기반에서 사이킷런과 유사한 API와 프레임워크를 제공
- 다양한 추천 알고리즘. 예를 들어 사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링을 쉽게 적용해 추천 시스템을 구축할 수 있음

In [2]:
pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 23.7 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633981 sha256=cf1753974d054859092e3cede0258a530f894c53d9002d9e7bf514b6160f7ca5
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


### Surprise를 이용한 추천 시스템 구축
- https://surpsrise.readthedocs.io/en/stable/ 참고

In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

- Surprise에서의 데이터로딩은 Dataset의 클래스를 이용해서만 가능함
- userId, movieId, rating과 같은 주요 데이터가 row 레벨 형태로 되어있는 포맷의 데이터만 처리함
- load_builtin() : 무비렌즈 사이트에서 제공하는 과거 버전 데이터 세트 로딩

In [5]:
data = Dataset.load_builtin('ml-100k') # 10만개 평점데이터
# 수행 시마다 동일하게 데이터를 내려받기 위해 random_state값 부여
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


- 한번 로컬 디렉터리에 데이터 저장 후에는 load_builtin 호출시 무비렌즈 사이트에 접속하지 않고 저장된 데이터 세트를 로딩
- Surprise는 자체적으로 로우 레벨의 데이터를 칼럼 레벨의 데이터로 변경하므로 원본인 로우 레벨의 사용자-아이템 평점 데이터를 데이터 세트로 적용해야 함

In [6]:
# SVD로 잠재 요인 협업 필터링 수행
# algo = SVD()와 같이 알고리즘 객체를 생성 후 fit(학습 데이터 세트)을 수행해 학습 데이터 세트 기반으로 추천 알고리즘을 학습
algo = SVD()
algo.fit(trainset)

In [8]:
predictions = algo.test(testset)
print('prediction type:', type(predictions), ' size:', len(predictions))
print( 'prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type: <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.755134317135357, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.9259829386686036, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.9464110125896705, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.636541760055754, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.405112926172189, details={'was_impossible': False})]

- 호출 결과는 리스트
- 호출 결과로 반환된 리스트 객체는 25000개의 Prediction 객체를 내부에 가짐
- Prediction 객체는 Surprise 패키지에서 제공하는 데이터 타입이며 개별 사용자 아이디(uid), 영화 아이디(iid)와 실제 평점 정보(r_ui)에 기반해 Surprise의 추천 예측 평점(est)데이터를 튜플 형태로 가짐
- Prediction 객체의 details 속성이 True라면 내부 처리시 추천 예측값을 생성할 수 없는 데이터라는 의미

In [9]:
# Prediction 객체에서 uid, iid, est 속성 추출
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.755134317135357),
 ('882', '291', 3.9259829386686036),
 ('535', '507', 3.9464110125896705)]

In [10]:
# 사용자의 아이템에 대한 추천 평점을 예측하는 predict() 메서드
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.30   {'was_impossible': False}


- 개별 사용자와 아이템 정보 입력시 추천 예측 평점을 est로 반환
- test()는 데이터세트의 모든 사용자와 아이템 아이디에 대해서 predict()를 반복적으로 수행한 결과

In [11]:
accuracy.rmse(predictions)

RMSE: 0.9511


0.9510524104101946

### Surprise 주요 모듈 소개

#### 1) Dataset
- user_id, item_id, rating 데이터가 로우레벨로 된 데이터 세트에만 적용 가능
- 네번째 칼럼부터는 아예 로딩을 수행하지 않고 제외함
- API
 - Dataset.load_builtin() : 무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터를 내려받음
 - Dataset.load_from_file(file_path, reader) : OS 파일에서 데이터를 로딩할 때 사용
 - Dataset.load_from_df(df, reader) : 판다스의 DataFrame에서 데이터를 로딩

#### 2) OS 파일 데이터를 Surprise 데이터 세트로 로딩
- Surprise에 OS 파일 로딩시 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있어서는 안됨

In [12]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')
# ratings_noh.csv 파일로 업로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('ratings_noh.csv', index = False, header = False)

- Reader 클래스 : 로딩될 데이터 파일의 파싱 포맷 정의

In [13]:
from surprise import Reader

reader = Reader(line_format = 'user item rating timestamp', sep = ',', rating_scale = (0.5, 5))
# 칼럼 헤더가 없고, 4개의 칼럼이 콤마로만 분리됨
# 각 필드의 칼럼명과 칼럼 분리 문자, 최소~최대 평점을 입력해 객체로 생성
data = Dataset.load_from_file('ratings_noh.csv', reader = reader)
# reader 객체를 참조해 데이터 파일 파싱하며 진행
# 앞의 3개 칼럼만 로딩되고 timestamp 칼럼은 제외됨

- Surpsrise 데이터 세트는 기본적으로 무비렌즈 데이터 형식을 따르므로 무비렌즈 데이터 형식이 아닌 다른 OS 파일의 경우 Reader 클래스를 먼저 설정해야 함
- Reader 클래스의 주요 생성 파라미터
 - line_format(string) : 칼럼을 순서대로 나열, 입력된 문자열을 공백으로 분리해 칼럼으로 인식
 - sep(char) : 칼럼을 분리하는 분리자, 디폴트는 \t
 - rating_scale (tuple, optional) : 평점 값의 최소 ~ 최대 평점을 설정, 디폴트는 (1, 5)

In [14]:
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
# n_factors : 잠재 요인 크기 K 값을 의미
algo = SVD(n_factors = 50, random_state = 0)

# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### 3) 판다스 DataFrame에서 Surpsrise 데이터 세트로 로딩
- DataFrame 역시 사용자 아이디, 아이템 아이디, 평점 칼럼의 형식을 지켜야 함

In [15]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale = (0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader) # 파라미터로 입력
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

algo = SVD(n_factors = 50, random_state = 0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## Surprise 추천 알고리즘 클래스
- Surprise에서 추천 예측을 위해 자주 사용되는 추천 알고리즘 클래스
1. SVD : 행렬 분해를 통핸 잠재 요인 협업 필터링을 위한 SVD 알고리즘
 - 비용함수 : 사용자 베이스라인 편향성을 감안한 평점 예측에 Regularization 적용
 - 입력 파라미터
    - n_factors : 잠재 요인 K의 개수. 커질수록 정확도가 높아질 수 있으나, 과적합 문제가 발생할 수 있음
    - n_epochs : SGD(확률적 경사하강법) 수행 시 반복 횟수, 디폴트 20
    - biased : 베이스라인 사용자 편향 적용 여부, 디폴트 True
2. KNNBasic : 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
3. BaselineOnly : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘
- 그외 https://surpsrise.readthedocs.io/en/stable/prediction_algorithms_package.html 참고
- 예측 성능 벤치마크 결과 : http://surpriselib.com/
 - SVD++알고리즘의 RMSE, MAE 성적이 가장 좋지만 상대적으로 시간이 너무 오래 걸려 데이터가 조금만 더 커져도 사용하기 어려울 것으로 보임
 - SVD, k-NN Baseline이 가장 성능 평가 수치가 좋음 
  - Baseline : 각 개인이 평점을 부여하는 성향을 반영해 평점을 계산

## 베이스라인 평점
- Baseline Rating: 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과
- 보통 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산
 - 전체 평균 평점 : 모든 사용자의 아이템에 대한 평점을 평균한 값
 - 사용자 편향 점수 : 사용자별 아이템 평점 평균 값 - 전체 평균 평점
 - 아이템 편향 점수 : 아이템별 평점 평균 값 - 전체 평균 평점

## 교차 검증과 하이퍼 파라미터 튜닝
- cross_validate() : 폴드된 데이터 세트의 개수와 성능측정방법을 명시해 교차 검증을 수행

In [16]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('ratings.csv')    # reading data in pandas df
reader = Reader(rating_scale = (0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state = 0)
# 알고리즘 객체, 데이터, 성능 평가 방법, 폴드 데이터 개수 객체로 입력
cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8723  0.8724  0.8816  0.8695  0.8752  0.8742  0.0041  
MAE (testset)     0.6699  0.6696  0.6753  0.6697  0.6712  0.6711  0.0022  
Fit time          4.15    3.97    4.57    3.96    4.02    4.13    0.23    
Test time         0.10    0.12    0.10    0.29    0.11    0.14    0.07    


{'test_rmse': array([0.87227182, 0.87240587, 0.88157693, 0.86948009, 0.87520966]),
 'test_mae': array([0.66993614, 0.66955838, 0.67533374, 0.66971094, 0.67117386]),
 'fit_time': (4.149383068084717,
  3.973351240158081,
  4.573880434036255,
  3.957689046859741,
  4.015574216842651),
 'test_time': (0.10131335258483887,
  0.1193842887878418,
  0.09818506240844727,
  0.2856006622314453,
  0.11205339431762695)}

- SVD의 경우 주로 SGD의 반복횟수를 지정하는 n_epochs와 SVD의 잠재요인 K의 크기를 지정하는 n_factors를 튜닝

In [17]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs' : [20, 40, 60], 'n_factors' : [50, 100, 200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mae로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures = ['rmse', 'mae'], cv = 3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8768122366285537
{'n_epochs': 20, 'n_factors': 50}


## Surprise를 이용한 개인화 영화 추천 시스템 구축
- Surprise로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은(관람하지 않은) 영화 중에서 개인 취향에 가장 적절한 영화를 추천
- Surprise는 데이터세트를 train_test_split()을 이용해 내부에서 사용하는 TrainSet 클래스 객체로 변환하지 않으면 fit()을 통해 학습할 수가 없음, 데이터세트를 그대로 fit()에 적용시 오류 발생

In [18]:
# 다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류가 발생
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors = 50, random_state = 0)
algo.fit(data)

AttributeError: ignored

- 데이터 세트 전체를 학습 데이터로 사용하려면 DatasetAutoFolds 클래스 이용
- DatasetAutoFolds 객체 생성 후 build_full_trainset() 메서드 호출시 전체 데이터를 학습 데이터 세트로 만들 수 있음

In [19]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format = 'user item rating timestamp', sep = ',', rating_scale = (0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file = 'ratings_noh.csv', reader = reader)

# 전체 데이터를 학습 데이터로 생성함
trainset = data_folds.build_full_trainset()

In [20]:
algo = SVD(n_epochs = 20, n_factors = 50, random_state = 0)
algo.fit(trainset)

- userId 9가 아직 평점을 매기지 않은 영화를 movieId 42로 선정한 뒤에 예측 평점 계산

In [21]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('movies.csv')

# userId = 9의 movieId 데이터를 추출해 movieId = 42 데이터가 있는지 확인
movieIds = ratings[ratings['userId'] == 9]['movieId']
if movieIds[movieIds == 42].count() == 0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
print(movies[movies['movieId'] == 42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [22]:
# 문자열 값으로 사용자 아이디와 아이템 아이디 입력해 예측값 계산
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose = True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


- 사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤에 예측 평점 순으로 영화를 추천

In [23]:
def get_unseen_surprise(ratings, movies, userId):
  # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

  # 모든 영화의 movieId를 리스트로 생성
  total_movies = movies['movieId'].tolist()

  # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수 :', len(seen_movies), '추천 대상 영화 수 :', len(unseen_movies),
        '전체 영화 수 :', len(total_movies))
  
  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수 : 46 추천 대상 영화 수 : 9696 전체 영화 수 : 9742


- 사용자 아이디 9는 전체 9742개 영화중 46개만 평점을 매겼으므로 나머지 영화 중 높은 예측 평점을 가진 순으로 영화 추천

In [24]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n = 10):
  # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
  # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
  # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
  def sortkey_est(pred):
    return pred.est

  # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출
  predictions.sort(key = sortkey_est, reverse = True)
  top_predictions = predictions[:top_n]

  # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [ int(pred.iid) for pred in top_predictions ]
  top_movie_rating = [ pred.est for pred in top_predictions ]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [ (id, title, rating) for id, title, rating in
                     zip(top_movie_ids, top_movie_titles, top_movie_rating) ]

  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n = 10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
  print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수 : 46 추천 대상 영화 수 : 9696 전체 영화 수 : 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
